## Sales Scraper for Loopnet.com

 A scraper dedicated to extracting information from Loopnet.com
 During running a brower will open, it's important that the browser opens with a proper resoltion so that all element are loaded.

## ⚠️WARNING⚠️

Overuse of this script may cause the provider to refuse the connection, either temporarily or permanently. To ensure safe usage:
- Avoid running the script continuously all day.
- Keep the randSleep function (present in the first cell) unchanged or increase its values for longer pauses.
- Only use when necessary
- This scrapper is kinda slow, and it's intented to be slow, as to avoid ban from the provider


## SETUP

1. install the dependecies
2. put the chromedriver on the drivers folder
3. all done


## USAGE

1. Run the first cell to do the setup. A browser will open, go to the desired search page of loopnet.com, something like `https://www.loopnet.com/search/commercial-real-estate/boerne-tx/for-sale/`, no extra random letters/symbosl/numbers at the end. 
2. Do not close the browser.



In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
import numpy as np
import random
import json
import pandas as pd


def randSleep():
    time.sleep(random.uniform(4, 6))


def pretty_print(d, indent=0):
    for key, value in d.items():
        print("  " * indent + str(key) + ":")
        if isinstance(value, dict):
            pretty_print(value, indent + 1)
        else:
            print("  " * (indent + 1) + str(value))




service = Service("./drivers/chromedriver")
driver = webdriver.Chrome()
driver.page_load_strategy = 'normal'  # Default behavior, waits for complete page load
wait = WebDriverWait(driver, 10) # mistery wait

driver.get("https://google.com/")




3. Run the next cell. A browser window will open and navigate through the listings. At the end of the process, a file named `loopnet_links.txt` will be created. This file contains a list of links from which the information will be extracted.

In [ ]:
link_list = []
output = open("loopnet_links.txt", "w")

current_page = driver.current_url
print("Current page: ", current_page)

pagenums = driver.find_elements(By.CSS_SELECTOR, ".paging.ng-isolate-scope")[0]
max_page = pagenums.text.split(" ")[-1]
max_page = int(max_page)

for page_num in range(1, max_page+1):
    
    URL = current_page  + "/" + str(page_num)
    if page_num != 1:
        driver.get(URL)
    
    randSleep()
    #placards = driver.find_elements(By.CSS_SELECTOR, ".placard")
    placards = driver.find_elements(By.CSS_SELECTOR, ".placard")
    link_num = 1
    for placard in placards:
        link = placard.find_elements(By.TAG_NAME, "a")
        hrefs = link[0].get_attribute("href")
        link_list.append(hrefs)
        print(page_num,link_num,hrefs)
        output.write(hrefs + "\n")
        link_num += 1
output.close()


print("Total links: ", len(link_list))

4. The next cell is used to load the file that was just saved, helping to avoid redundant runs. If you are not loading from file, e.g. running everthing by sequence, you may skip this cell.

for testing purposes the current file is `loopnet_links_safe.txt` which contains 500 links.

In [ ]:
link_list = np.loadtxt("loopnet_links.txt", dtype=str)

print("Total links: ", len(link_list))

5. The following cell opens each link from the list in the file, extracts all relevant information from each one, and saves the data into a `.JSON` file named `loopnet_data.json`. You may also change how many links are scraped by changing `init_page` and `final_page`, if one wishes to do scrapping by parts. When running this part, a new window will appear, it needs to be `1980x1080` to ensure that all properties of the page are properly loaded.
   
Remember, here 0 index corresponds to the 1st page, 1 to the 2nd and so on.

In [9]:
# these variables are as it its so it's possible to pause and resume the script

all_data = {}
i = 0

In [35]:
# Now the proper data scrapping

final_page = len(link_list)

# DONE HERE
def get_title(): # Tittle info OK
    title = driver.find_elements(By.TAG_NAME, "h1")
    title = title[0].text # INFO
    return title

# DONE HERE
def get_footnote(): # footnoot info, aways present
    footnote_infos = driver.find_elements(By.CLASS_NAME, "property-timestamp")
    footnote_infos = footnote_infos[0].find_elements(By.TAG_NAME, "h4")
    listing_id = footnote_infos[0].text.split(":")[-1] # INFO
    market_date = footnote_infos[1].text.split(":")[-1] # INFO
    last_update = footnote_infos[2].text.split(":")[-1] # INFO
    address = footnote_infos[3].text.split(":")[-1] # INFO
    return listing_id, market_date, last_update, address


# DONE HERE?
def get_facts(): # facts info (optional)
    facts = driver.find_elements(By.CSS_SELECTOR, ".listing-features")
    if len(facts) == 0:
        return {}
    
    facts_names = facts[0].find_elements(By.CSS_SELECTOR, ".fact-name")
    vals = facts[0].find_elements(By.CSS_SELECTOR, ".property-facts__data-item-text ")
    dic_facts = {}
    for i in range(0, len(facts_names)):
        dic_facts[facts_names[i].text] = vals[i].text

    return dic_facts

def get_amenites():
    amenites_all = driver.find_elements(By.CSS_SELECTOR, ".highlights.features-and-amenities.listing-features_amenities")
    features = []
    if len(amenites_all) != 0:
        for list_part in amenites_all:
            amenities = list_part.find_elements(By.TAG_NAME, "li")
            for amenity in amenities:
                features.append(amenity.text)
    features = ", ".join(features)
    return features

# DONE HERE
def get_tax():
    tax = {}
    tax_all = driver.find_elements(By.ID, "taxes")
    if len(tax_all) == 0:
        return tax
    infos = tax_all[0].find_elements(By.TAG_NAME, "td")
    for i in range(0, len(infos)):
        if i % 2 == 0:
            tax[infos[i].text] = infos[i+1].text
    return tax


# DONE HERE
def get_zoning():
    zoning = {}
    zoning_all = driver.find_elements(By.CSS_SELECTOR, ".taxes-zoning.taxes-zoning__zoning")
    if len(zoning_all) == 0:
        return zoning
    infos = zoning_all[0].find_elements(By.TAG_NAME, "td")
    for i in range(0, len(infos)):
        if i % 2 == 0:
            zoning[infos[i].text] = infos[i+1].text
    return zoning

def get_demographics1():
    demographics = []
    demographics_all = driver.find_elements(By.CSS_SELECTOR, ".demographics-industrial__table-one")
    if len(demographics_all) == 0:
        return demographics
    else:
        infos = demographics_all[0].find_elements(By.CLASS_NAME, "trade-area-table")
        for i in range(1, len(infos)):
            demographics.append(infos[i].text.replace("\n", " "))
    return demographics

# DONE HERE
def get_brokers():
    names = []
    phones = []
    brokers_infos = driver.find_elements(By.CSS_SELECTOR, ".broker-bio") # INFO
    if brokers_infos != []:
        for i in range(len(brokers_infos)):
            name = brokers_infos[i].find_elements(By.CLASS_NAME, "header-text")[0].text
            phone = brokers_infos[i].find_elements(By.CLASS_NAME, "broker-phone")[0].text
            if phone == "":
                phone = "Not available"
            else:
                phones.append(phone)
            names.append(name)

    else:
        brokers_infos = driver.find_element(By.CSS_SELECTOR, ".container-contact-form ")
        name = brokers_infos.find_elements(By.CSS_SELECTOR, ".contact-name")
        if name == []:
            name = "Not available"
        else:
            name = name[0].text
        phone = brokers_infos.find_elements(By.CSS_SELECTOR, ".phone-number")
        if phone == []:
            phone = "Not available"
        else:
            phones.append(phone[0].text)
        names.append(name)

    return names, phones

def get_labor():
    demographics = {}
    demographics_all = driver.find_elements(By.CSS_SELECTOR, ".demographics-industrial__table-two")
    if len(demographics_all) == 0:
        return demographics
    
    else:
        row_names = demographics_all[0].find_elements(By.CLASS_NAME, "row-label")
        table_data = demographics_all[0].find_elements(By.CLASS_NAME, "table-data")
        for i in range(1, len(row_names)):
            demographics[row_names[i].text] = table_data[i].text
    return demographics

def get_nearby_am(): # need a quick fix
    # check for hotels:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    pois = {}
    near_interests = driver.find_elements(By.CSS_SELECTOR, ".highlights.include-in-page.poi ")
    if len(near_interests) == 0:
        return []

    for poi in near_interests:
        name = poi.find_elements(By.TAG_NAME, "h3")
        places = []
        for place in poi.find_elements(By.TAG_NAME, "tr"):
            places.append(place.text)
        pois[name[0].text] = places[1:]
    return pois

def get_utilities():
    utilities = []
    utilities_all = driver.find_elements(By.CSS_SELECTOR, ".highlights.include-in-page.utilities")
    if len(utilities_all) == 0:
        return utilities
    infos = utilities_all[0].find_elements(By.TAG_NAME, "li")
    for i in range(0, len(infos)):
        utilities.append(infos[i].text)
    
    return utilities

def get_spaces():
    ## Fix to add zoning
    spaces = {}
    # how many spaces cards there are
    spaces_table = driver.find_elements(By.CSS_SELECTOR, ".available-spaces.available-spaces--desktop.available-spaces--cards")
    print(len(spaces_table))
    # loop throught the cards

    expand_spaces = driver.find_elements(By.CSS_SELECTOR, ".available-spaces__view-more")
    if expand_spaces != []:
        driver.execute_script("arguments[0].scrollIntoView();", expand_spaces[0])
        time.sleep(1)
        expand_spaces[0].click()
        time.sleep(1)
        


    if expand_spaces != []:
        for i in range(0, len(expand_spaces)):
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", expand_spaces[i])
            driver.execute_script("window.scrollBy(0, -500);")
            time.sleep(1)
            expand_spaces[i].click()
            time.sleep(1)

def get_spaces():
    ## Fix to add zoning
    spaces = {}
    # how many spaces cards there are
    spaces_table = driver.find_elements(By.CSS_SELECTOR, ".available-spaces.available-spaces--desktop.available-spaces--cards")
    print(len(spaces_table))
    # loop throught the cards

    expand_spaces = driver.find_elements(By.CSS_SELECTOR, ".available-spaces__view-more-wrap")

    if expand_spaces != []:
        for i in range(0, len(expand_spaces)):
            driver.execute_script("arguments[0].scrollIntoView();", expand_spaces[i])
            driver.execute_script("window.scrollBy(0, -500);")
            time.sleep(2)
            expand_spaces[i].click()
            time.sleep(2)
        
    if len(spaces_table) > 0:

        # loop for each major table
        for i in range(len(spaces_table)):
            print(i,len(spaces_table))
            # scroll everthing into view
            driver.execute_script("arguments[0].scrollIntoView();", spaces_table[i])
            driver.execute_script("window.scrollBy(0, -100);")
            # data for the card, as well as the facts
            this_space = {}
            this_space['address_space'] = spaces_table[i].find_elements(By.CSS_SELECTOR, ".as-space-info__address")[0].text
            fact_names = spaces_table[i].find_elements(By.CSS_SELECTOR, ".fact-name")
            fact_vals = spaces_table[i].find_elements(By.CSS_SELECTOR, ".property-facts__data-item-text ")
            for j in range(len(fact_names)):
                this_space[fact_names[j].text] = fact_vals[j].text
            this_space["zooning_space"] = spaces_table[i].find_elements(By.CSS_SELECTOR, ".property-facts__data-item-text ")[-1].text


            # get the commom property of all the rooms from the header.
            floors_basic_prop = spaces_table[i].find_elements(By.CSS_SELECTOR, ".available-spaces__header-headings")
            floors_basic_prop = floors_basic_prop[0].text.split("\n")

            # fin the listing of all the props.
            floors = spaces_table[i].find_elements(By.CSS_SELECTOR, ".available-spaces__content")
            #floors_basic_vals = floors[0].text.split("\n")

            for j in range(0,len(floors)):
                floors[j].click()
                time.sleep(1)
                this_floor = {}
                for k in range(0,len(floors_basic_prop)):
                    this_floor[floors_basic_prop[k].lower()] = floors[j].text.split("\n")[k].lower()
                    #print(floors_basic_prop[k], floors[j].text.split("\n")[k])
                extras = floors[j].find_elements(By.CSS_SELECTOR, ".available-spaces__highlights-wrap")
                if extras != []:
                    extras = extras[0].text.split("\n")
                    extras = ", ".join(extras)
                    this_floor["higlights"] = extras
                
                this_space["floor-room_" + str(j)] = this_floor

            spaces["space_" + str(i)] = this_space

        return spaces

    else:
        print("No cards")
        this_floor = {}
        tables = driver.find_elements(By.CSS_SELECTOR, ".available-spaces.available-spaces--desktop")
        
        if len(tables) == 0:
            return spaces
        
        print("tables: ",len(tables))

        floors = tables[0].find_elements(By.CSS_SELECTOR, ".available-spaces__content")
        # get basic property of the listing
        
        floors_basic_prop = tables[0].find_elements(By.CSS_SELECTOR, ".available-spaces__header")
        if len(floors_basic_prop) == 0:
            return spaces
        
        # basic props of each floor/room     
        floors_basic_prop = floors_basic_prop[0].text.split("\n")

        # fint all the floors
        floors = tables[0].find_elements(By.CSS_SELECTOR, ".available-spaces__content")

        for j in range(0,len(floors)):
            # fix the view range
            if j == 0:
                driver.execute_script("arguments[0].scrollIntoView();", floors[j])
                time.sleep(1)
                driver.execute_script("window.scrollBy(0, -300);")
                time.sleep(1)

            floors[j].click()
            time.sleep(1)
            this_floor = {}
            for k in range(0,len(floors_basic_prop)):
                this_floor[floors_basic_prop[k].lower()] = floors[j].text.split("\n")[k].lower()
                #print(floors_basic_prop[k], floors[j].text.split("\n")[k])
            extras = floors[j].find_elements(By.CSS_SELECTOR, ".available-spaces__highlights-wrap")
            if extras != []:
                extras = extras[0].text.split("\n")
                extras = ", ".join(extras)
                this_floor["higlights"] = extras


            spaces["space_" + str(j)] = this_floor

        return spaces        


print("Total links: ", len(link_list))
print("Links:")


driver.set_window_size(1980, 1080)

while i < len(link_list):
    data = {}
    print("c: ", i, link_list[i])
    driver.get(link_list[i])
    randSleep()

    data["link"] = link_list[i] # link for the ad
    data["title"] = get_title() # title of the ad
    data["brokers"], data["phones"] = get_brokers() # broker info
    list_id, market_date, last_update, address = get_footnote()
    data["list ID"] = list_id # listing ID
    data["market date"] = market_date # market date
    data["last update"] = last_update # last update
    data["address"] = address # address
    data["facts"] = get_facts() # facts
    data["amenities"] = get_amenites() # amenities
    data["utilities"] = get_utilities() # utilities
    data["tax"] = get_tax() # tax
    data["zoning"] = get_zoning() # zoning
    data["spaces"] = get_spaces() # spaces (it will create lots of extra fields on the table)
    data["labor"] = get_labor() # labor
    data["demographics"] = get_demographics1() # demographics
    data["poi"] = get_nearby_am() # nearby amenities
    all_data[i] = data
    print("Data:")
    pretty_print(data)
    i +=1

print("Done \a")

output = open("loopnet_data.json", "w")
json.dump(all_data, output, indent=4)
output.close()
    

Total links:  500
Links:
c:  187 https://www.loopnet.com/Listing/118-N-Seguin-Rd-Converse-TX/31764864/
0
No cards
Data:
link:
  https://www.loopnet.com/Listing/118-N-Seguin-Rd-Converse-TX/31764864/
title:
  118 North Seguin Road - Converse 118 N Seguin Rd
brokers:
  ['Laurence Seiterle']
phones:
  []
list ID:
   31764864
market date:
   6/5/2024
last update:
   11/25/2024
address:
   118 N Seguin Rd, Converse, TX 78109
facts:
amenities:
  
utilities:
  []
tax:
zoning:
spaces:
labor:
demographics:
  []
poi:
  []
c:  188 https://www.loopnet.com/Listing/3015-San-Pedro-Ave-San-Antonio-TX/31695725/
0
No cards
tables:  1
Data:
link:
  https://www.loopnet.com/Listing/3015-San-Pedro-Ave-San-Antonio-TX/31695725/
title:
  3015 San Pedro Ave
brokers:
  ['Eric Hanken']
phones:
  []
list ID:
   31695725
market date:
   12/5/2024
last update:
   1/6/2025
address:
   3015 San Pedro Ave, San Antonio, TX 78212
facts:
  Building Type:
    Office
  Year Built/Renovated:
    1979/2019
  Building Height:
 

6. The following cell converts the `.JSON` data into a table (or DataFrame) containing all the extracted information, without filtering by attributes. It also prints all the possible fields that may be of interest for saving. If a field is blank, the information was not available.

In [3]:
# Table generator

all_data = json.load(open("loopnet_data_safe.json"))

for entry in all_data.values():
    if 'spaces' in entry:
        entry['spaces'] = ", ".join([f"{k}: {v}" for k, v in entry['spaces'].items()])

df = pd.json_normalize(all_data.values())
print("DataFrame created with shape:", df.shape)
print("Columns:", df.columns)

#df = df.applymap(lambda x: ", ".join(x) if isinstance(x, list) else x)

df.to_excel("all_loopnet_data.xlsx", index=False)



FileNotFoundError: [Errno 2] No such file or directory: 'loopnet_data.json'

With all the data scraped and secure, this next cell servers to save a filtered table, if the one may desire.

7. To do so, change the words on the array `my_fields`, so it may look something like `my_fields = ["link", "title", "brokers.names", "brokers.phones"]`.
8. Run the cell, and a new excel file with name `filtered_loopnet_data.xlsx` will appear, this one contais only the filtered data defined by `my_fields`

In [43]:
my_fields = ["link", "title","brokers"]

df_to_save = df[my_fields]
print("Filtered infos:")
print(df_to_save)
df_to_save.to_excel("filtered_loopnet_data.xlsx", index=False)


Filtered infos:
                                                  link  \
0    https://www.loopnet.com/Listing/5930-6010-Eise...   
1    https://www.loopnet.com/Listing/1228-Cornerway...   
2    https://www.loopnet.com/Listing/542-SE-Loop-41...   
3    https://www.loopnet.com/Listing/10000-Rogers-R...   
4    https://www.loopnet.com/Listing/7001-San-Anton...   
..                                                 ...   
495  https://www.loopnet.com/Listing/14402-Blanco-R...   
496  https://www.loopnet.com/Listing/300-Convent-St...   
497  https://www.loopnet.com/Listing/8527-Village-D...   
498  https://www.loopnet.com/Listing/10942-Wye-Dr-S...   
499  https://www.loopnet.com/Listing/19026-Ridgewoo...   

                                              title  \
0    Eisenhauer Business Park San Antonio, TX 78218   
1          Cubework San Antonio 1228 Cornerway Blvd   
2                       Oakmont 410 542 SE Loop 410   
3                                  10000 Rogers Run   
4     Balcon